# Build Network Graph
Borrowed code from the earlier notebooks

In [275]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import seaborn as sns
import warnings
import random # NEW! For Agents
warnings.filterwarnings('ignore', category=UserWarning)

np.random.seed(99) # Set random seed for reproducibility

df = pd.read_csv('../data/bilat_mig.csv')# Load the migration data
# Rebuild the directed graph
G = nx.DiGraph()
for index, row in df.iterrows():
    if row['da_pb_closed'] > 0:
        G.add_edge(row['orig'], row['dest'], weight=row['da_pb_closed'])
print("Libraries loaded and graph rebuilt successfully!")
print(f"Graph: {G.number_of_nodes()} countries, {G.number_of_edges()} migration flows")

Libraries loaded and graph rebuilt successfully!
Graph: 10 countries, 88 migration flows


## Test Traversing Graph
Test of different ways you can set parameters onto nodes.\
Can be used later to set GDP ect.\
If you add a parameter to one node the other nodes all remain empty.

In [276]:
print(G.nodes)

# Set node parameters
G.nodes["USA"]["GDP"] = 1000

# Test retrieving data
print(G.nodes["USA"]["GDP"])

# Error if not set yet. Parameters are node-by-node
try:
    print(G.nodes["IND"]["GDP"])
except KeyError:
    print("Parameter not found")


# Test retrieving random
print("Random Node:",random.choice(list(G.nodes)))

['IND', 'PHL', 'POL', 'GBR', 'DEU', 'MEX', 'CAN', 'USA', 'AUS', 'SYR']
1000
Parameter not found
Random Node: MEX


# Modelling
## Agent Setup
Start with a simple agent that can traverse a networkX network.\
Uses template from wk 7 Lab.\
Agents will have all the attributes for a single person, as well as their decision making methods.

### Cost Functions

Runs using two cost functions.\
**check_if_move** tests if the agent even wants to move\
**check_where_move** tests where the agent would go if it does want to move.

NOTE: This is not the only way of doing this. You could have one cost function that tests the likelihood of moving to a specific, rather than general likelihood of moving

### Other Notes

Display gives the option to turn off print statements later.


TO DO:\
Swap random check_move for something more robust\
Replace getting a random new position with graph based

You could definitely combine Move() and Checkmove()

In [277]:
class Agent:
    """
    Summary of main attributes:
    id              - The "name" of this agent
    birth_place     - The source of this agent
    living_place    - The current living place
    job             - Fill later
    age             - Fill later
    wanderlust      - threshold to move per step
    locations       - A list of all locations lived in. If it doesn't move, a duplicate is added
    """
    

    def __init__(self, id, birth_node):
        """Creates a new agent at the given location."""
        self.birth_place = birth_node
        self.living_place = birth_node
        self.job = "Builder"
        self.age = 0
        self.wanderlust = 0.3
        self.id = id
        self.locations = []

    # ------ Cost Functions -------
    def check_if_move(self, world):
        """
        First cost function. Check if current living conditions make it worth moving.
        World currently unused but likely in future iterations.
        Returns either true or false.
        """
        if(random.random() < self.wanderlust):
            return True
        else:
            return False

    def check_where_move(self, world):
        """
        Second cost function. If this agent moves, where would it move to?
        Returns a new connected location        
        """

        # Check valid neighbours
        possible_countries = world.get_neighbours(self.living_place)
        # print("Possible Countries:", list(possible_countries))

        # Get a random one for now
        new_country = random.choice(list(possible_countries))

        return new_country
    


    def step(self, world, display=False):
        self.age += 1

        if(self.check_if_move(world)):
            new_country = self.check_where_move(world)

            # Printouts
            if(display): 
                print("Checking", self.id, 
                "... I live in ", self.living_place, 
                "and I hate it. I'm moving to:", new_country)
        
        else:
            new_country = self.living_place

            # Printouts
            if(display): 
                print("Checking", self.id, 
                "... I live in ", self.living_place, 
                "and I'm happy here")
            
        
        # Add the new location to the array
        self.living_place = new_country
        self.locations.append(new_country)

## World Setup
Create a world environment to run and control enviro params.\
The world contains links to all the nodes and agents in it.\
But the decision making is controlled by the agent, not the world.\

Later the world will contain parameters about each node.

In [278]:
class World:
    """
    Summary of main attributes:
    houses? wealth? overpopulation?
    """

    def __init__(self, world_graph):
        self.world_graph = world_graph
        self.agents      = []

    def get_rand_country(self):
        random_item = random.choice(list(self.world_graph.nodes))
        return random_item
    
    def get_neighbours(self, node):
        """Convenience method that looks neater"""
        return self.world_graph.neighbors(node)

    def add_agent(self):
        new_id = len(self.agents) + 1
        rand_birth_place = self.get_rand_country()
        new_agent = Agent(new_id, rand_birth_place)

        print("Adding new agent born in ", rand_birth_place)
        self.agents.append(new_agent)

    def get_agents(self):
        return self.agents
    
    def step(self):
        for agent in self.agents:
            agent.step(self, display=True) # Give the whole object so it can decision make

In [279]:
# Configure new world
random.seed(99) # Seed for this specific set of operations

world = World(G)

# Add 3 agents
world.add_agent()
world.add_agent()
world.add_agent()

# Run 5 generations and see where they end up
for step in range(5):
    print("----- Generation", step, "-----")
    world.step()

    # # ask the agents again
    # for agent in world.get_agents():
    #     print("Hi, I'm", agent.id,"and I live in", agent.living_place)
    
    print("------------------------")


# Check the route for each agent
print("Born |   0  |   1  |   2  |   3  |   4  |")
for agent in world.get_agents():
    print("  ", agent.id, agent.locations)

Adding new agent born in  CAN
Adding new agent born in  CAN
Adding new agent born in  GBR
----- Generation 0 -----
Checking 1 ... I live in  CAN and I'm happy here
Checking 2 ... I live in  CAN and I hate it. I'm moving to: SYR
Checking 3 ... I live in  GBR and I'm happy here
------------------------
----- Generation 1 -----
Checking 1 ... I live in  CAN and I hate it. I'm moving to: MEX
Checking 2 ... I live in  SYR and I'm happy here
Checking 3 ... I live in  GBR and I'm happy here
------------------------
----- Generation 2 -----
Checking 1 ... I live in  MEX and I hate it. I'm moving to: USA
Checking 2 ... I live in  SYR and I hate it. I'm moving to: DEU
Checking 3 ... I live in  GBR and I'm happy here
------------------------
----- Generation 3 -----
Checking 1 ... I live in  USA and I'm happy here
Checking 2 ... I live in  DEU and I'm happy here
Checking 3 ... I live in  GBR and I'm happy here
------------------------
----- Generation 4 -----
Checking 1 ... I live in  USA and I h

# Jobs
The below structure extends and replaces the existing world class definition.\
This allows reusing all the above methods while focussing on the new additions\

The below idea attempts to setup a simple job market idea.

## Job Types
6 job types:
- Health Care (Nurse)
- Retail ()
- Admin
- Service (Bartender)
- Science (Scientist)
- Construction (Builder)
Based on: https://www.abs.gov.au/statistics/labour/jobs/jobs-australia/latest-release#industry

## Job array
Job Name\

Job Demand - Generic score from 0 to 1 to represent how good it is to have that job here.
Initialise as random number. Then update every step

Job Workers: How many people are working this job

Job Slots - can expand later to only allow certain number of people

In [ ]:
# class World:
class World(World): # Extend the existing world object

    def __init__(self, world_graph):
        # Add list of agents
        self.agents      = []

        # --- NEW: Add and initialise job lists ---
        for node in world_graph:
            self.build_job_list(node)
        
        # Add the world graph
        self.world_graph = world_graph

    # --- NEW: Job methods ---

    def build_job_list(node):
        """Initialise the job dataframe and add it to a node."""
        jobs = pd.DataFrame([])
        jobs["Job Name"]   = [        "Nurse",        "Retail",         "Admin",     "Bartender",     "Scientist",       "Builder"]
        jobs["Job Demand"] = [random.random(), random.random(), random.random(), random.random(), random.random(), random.random()]
        jobs["Job Workers"]= [              0,               0,               0,               0,               0,               0]

        # Save the entire dataframe as a node attribute
        node["job"] = jobs
    

    def get_jobs(node):
        """Helper method. Retrieve the job array"""
        return node["job"]

    
    def step(self):


        for agent in self.agents:
            agent.step(self, display=True) # Give the whole object so it can decision make

        # --- NEW: Update Job Environment ---
        for node in self.world_graph:
            self.get_jobs(node)

Also need to update agent decision making

In [ ]:
class Agent(Agent): # Extend the existing agent class
    """
    Summary of main attributes:
    id              - The "name" of this agent
    birth_place     - The source of this agent
    living_place    - The current living place
    job             - Fill later
    age             - Fill later
    wanderlust      - threshold to move per step
    locations       - A list of all locations lived in. If it doesn't move, a duplicate is added
    """
    

    def __init__(self, id, birth_node):
        """Creates a new agent at the given location."""
        self.birth_place = birth_node
        self.living_place = birth_node
        self.job = "Builder"
        self.age = 0
        self.wanderlust = 0.3
        self.id = id
        self.locations = []

    # ------ Cost Functions -------
    def check_if_move(self, world):
        """
        First cost function. Check if current living conditions make it worth moving.
        World currently unused but likely in future iterations.
        Returns either true or false.
        """
        if(random.random() < self.wanderlust):
            return True
        else:
            return False

    def check_where_move(self, world):
        """
        Second cost function. If this agent moves, where would it move to?
        Returns a new connected location        
        """

        # Check valid neighbours
        possible_countries = world.get_neighbours(self.living_place)
        # print("Possible Countries:", list(possible_countries))

        # Get a random one for now
        new_country = random.choice(list(possible_countries))

        return new_country
    


    def step(self, world, display=False):
        self.age += 1

        if(self.check_if_move(world)):
            new_country = self.check_where_move(world)

            # Printouts
            if(display): 
                print("Checking", self.id, 
                "... I live in ", self.living_place, 
                "and I hate it. I'm moving to:", new_country)
        
        else:
            new_country = self.living_place

            # Printouts
            if(display): 
                print("Checking", self.id, 
                "... I live in ", self.living_place, 
                "and I'm happy here")
            
        
        # Add the new location to the array
        self.living_place = new_country
        self.locations.append(new_country)

In [ ]:
random.seed(99) # Seed for this specific set of operations

world = World(G)

# Add 3 agents
world.add_agent()
world.add_agent()
world.add_agent()

# Natural Disasters / War

Both natural disasters and war are large events that affect almost all people within a country.\
Here they are modelled as random "Events". In the real world these events are not random, and have identifiable traits in the leadup to an event.\

Severity: The size of the negative modifier

Extensiveness: How many people does this affect in the country. As represented by a percentage of people.
This is not a perfect representation as disasters and war are usually localised on a single city, but we don't have the capacity to model that under the current setup.